# Project Case:
### Olist Store is the largest department store in Brazilian marketplace
Olist connects small businesses from all over Brazil to channels without hassle and with a single contract The Brazilian
ecommerce public dataset of orders (from 2016 to 2018 made at Olist Store is provided to your
company for analysis

Your manager is asking you to critically analyse the provided datasets using Business Intelligence
tools and provide some marketing findings recommendations in a report format The dataset has
information of 100 k orders made at multiple marketplaces in Brazil Its features allow viewing an
order from multiple dimensions from order status, price, payment and freight performance to
customer location, product attributes and finally reviews written by customers A geolocation
dataset that relates Brazilian zip codes to lat lng coordinates is also integrated in the dataset

After a customer purchases the product from Olist Store, a seller gets notified to fulfill that order
Once the customer receives the product, or the estimated delivery date is due, the customer gets
a satisfaction survey by email where they can give a note for the purchase experience and write
down some comments

# Olist Geolocation Dataset 
In this notebook, we clean and transform the geolocation dataset provided by Olist.

## Background:
###  CEP: the Brazilian Zip Code
A brazilian zip code, also know as CEP, stands for Postal Adressing Code (Código de Endereçamento Postal) and contains 8 digits. Introduced in 1972 as a sequence of five digits, it was expanded to eight digits in 1992 to allow for more precise localization. The standard format is "nnnnn-nnn" (the original five digits, an hyphen, and the new three digits).

**CEP**: 12.345-678

Most cities with population around 100,000 and above have a CEP assigned to every public place and to some high-occupancy private spaces, like major commercial buildings and large residential condos. Small towns are assigned a general 5-digit code followed by the suffix -000.

1. the first part is composed by 5 digits that represent Region, Subregion, Sector, Subsector and Subsector Splitter.
2. the second part contain 3 digits, separated by an hyphen from the first, and it represents the Distribution Identifiers.

More info here: https://www.correios.com.br/a-a-z/cep-codigo-de-enderecamento-postal

### Geolocation dataset:
The first 5 digits of the zip code are provided, together with latitude and longitude information, city name and state name.

#### Issues:
- Since we only have the first 5 digits of the zip code, named zip_code_prefix, precise location cannot be determined. This also explains why there are varying lat/lng information for each zip_code_prefix.
- Portugese characters include characters which are represented differently from english characters. City names are not uniformly respresented in a single way in the dataset. This is the main challenge to clean up and transform.

#### Approach:
- Find a reliable source of City, State Names together with CEP information where available, and use this to clean/transform the names in the dataset

## Begin: Import necessary libraries

In [1]:
import os
import json
import urllib
import requests
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv
from datetime import date

In [2]:
# Load environment variables from .env file
load_dotenv()

# Access environment variable
db_local=os.environ['olist_db_local']
db_azure=os.environ['olist_db_azure']
back4app_id=os.environ['back4app_id']
back4app_key=os.environ['back4app_key']
export_path_local=os.environ['export_path_local']

## A source of CEP data with city and state names, and other info, is available on back4app
We write an API call to retrieve the data

In [47]:
# API call to back4app to find the total count of Brazil CEP
where = urllib.parse.quote_plus("""
{
    "CEP": {
        "$exists": true
    }
}
""")
url = 'https://parseapi.back4app.com/classes/Brazilzipcode_CEP?count=1&limit=0&where=%s' % where
headers = {
    'X-Parse-Application-Id': back4app_id,
    'X-Parse-REST-API-Key': back4app_key
}
data = json.loads(requests.get(url, headers=headers).content.decode('utf-8'))
print(json.dumps(data, indent=2))
count_value = data["count"]

{
  "results": [],
  "count": 730321
}


In [5]:
# API call to back4app to retrieve all Brazil CEP
# Since there is 730,321 entries, we retrieve in chunks at 1000 rows each page
# Save each page as a .json file

page_size = 1000
for i in range(0, count_value, page_size):
    url = f'https://parseapi.back4app.com/classes/Brazilzipcode_CEP?skip={i}&limit={1000}&order=CEP&excludeKeys=info,logradouro,numero'
    headers = {
        'X-Parse-Application-Id': back4app_id,
        'X-Parse-REST-API-Key': back4app_key
    }
    data = json.loads(requests.get(url, headers=headers).content.decode('utf-8'))
    filename = f'{export_path_local}page_{i // page_size + 1}.json'
    with open(filename, 'w') as outfile:
        json.dump(data, outfile, indent=4)
    print(f"Saved data from page {i // page_size + 1} to {filename}")

print("All pages retrieved and saved successfully.")

Saved data from page 1 to page_1.json
Saved data from page 2 to page_2.json
Saved data from page 3 to page_3.json
Saved data from page 4 to page_4.json
Saved data from page 5 to page_5.json
Saved data from page 6 to page_6.json
Saved data from page 7 to page_7.json
Saved data from page 8 to page_8.json
Saved data from page 9 to page_9.json
Saved data from page 10 to page_10.json
Saved data from page 11 to page_11.json
Saved data from page 12 to page_12.json
Saved data from page 13 to page_13.json
Saved data from page 14 to page_14.json
Saved data from page 15 to page_15.json
Saved data from page 16 to page_16.json
Saved data from page 17 to page_17.json
Saved data from page 18 to page_18.json
Saved data from page 19 to page_19.json
Saved data from page 20 to page_20.json
Saved data from page 21 to page_21.json
Saved data from page 22 to page_22.json
Saved data from page 23 to page_23.json
Saved data from page 24 to page_24.json
Saved data from page 25 to page_25.json
Saved data from pa

Saved data from page 202 to page_202.json
Saved data from page 203 to page_203.json
Saved data from page 204 to page_204.json
Saved data from page 205 to page_205.json
Saved data from page 206 to page_206.json
Saved data from page 207 to page_207.json
Saved data from page 208 to page_208.json
Saved data from page 209 to page_209.json
Saved data from page 210 to page_210.json
Saved data from page 211 to page_211.json
Saved data from page 212 to page_212.json
Saved data from page 213 to page_213.json
Saved data from page 214 to page_214.json
Saved data from page 215 to page_215.json
Saved data from page 216 to page_216.json
Saved data from page 217 to page_217.json
Saved data from page 218 to page_218.json
Saved data from page 219 to page_219.json
Saved data from page 220 to page_220.json
Saved data from page 221 to page_221.json
Saved data from page 222 to page_222.json
Saved data from page 223 to page_223.json
Saved data from page 224 to page_224.json
Saved data from page 225 to page_2

Saved data from page 398 to page_398.json
Saved data from page 399 to page_399.json
Saved data from page 400 to page_400.json
Saved data from page 401 to page_401.json
Saved data from page 402 to page_402.json
Saved data from page 403 to page_403.json
Saved data from page 404 to page_404.json
Saved data from page 405 to page_405.json
Saved data from page 406 to page_406.json
Saved data from page 407 to page_407.json
Saved data from page 408 to page_408.json
Saved data from page 409 to page_409.json
Saved data from page 410 to page_410.json
Saved data from page 411 to page_411.json
Saved data from page 412 to page_412.json
Saved data from page 413 to page_413.json
Saved data from page 414 to page_414.json
Saved data from page 415 to page_415.json
Saved data from page 416 to page_416.json
Saved data from page 417 to page_417.json
Saved data from page 418 to page_418.json
Saved data from page 419 to page_419.json
Saved data from page 420 to page_420.json
Saved data from page 421 to page_4

Saved data from page 594 to page_594.json
Saved data from page 595 to page_595.json
Saved data from page 596 to page_596.json
Saved data from page 597 to page_597.json
Saved data from page 598 to page_598.json
Saved data from page 599 to page_599.json
Saved data from page 600 to page_600.json
Saved data from page 601 to page_601.json
Saved data from page 602 to page_602.json
Saved data from page 603 to page_603.json
Saved data from page 604 to page_604.json
Saved data from page 605 to page_605.json
Saved data from page 606 to page_606.json
Saved data from page 607 to page_607.json
Saved data from page 608 to page_608.json
Saved data from page 609 to page_609.json
Saved data from page 610 to page_610.json
Saved data from page 611 to page_611.json
Saved data from page 612 to page_612.json
Saved data from page 613 to page_613.json
Saved data from page 614 to page_614.json
Saved data from page 615 to page_615.json
Saved data from page 616 to page_616.json
Saved data from page 617 to page_6

In [3]:
# Parse the .json files and combine into one dataframe

# Define the page size and the range for your loop
page_size = 1000
num_pages = i // page_size + 1
all_data = pd.DataFrame()

# Loop through the number of pages you have
for n in range(num_pages):
    # Read each JSON file
    filename = f'{export_path_local}page_{n+1}.json'
    with open(filename, 'r') as file:
        data = json.load(file)
        # Convert the JSON data to a DataFrame
        df = pd.DataFrame(data['results'])  # Update 'results' if your data has a different key
        # Append the DataFrame to the all_data DataFrame
        all_data = pd.concat([all_data, df], ignore_index=True)

# Now `all_data` contains all the data from the JSON files in one DataFrame
all_data.describe()

,objectId,CEP,cidade,estado,bairro,createdAt,updatedAt
count,730321,730321,730321,730321,730321,730321,730321
unique,730321,730261,10162,73,18926,51631,51631
top,b74XHFEeVf,70830016,São Paulo,SP,Centro,2019-12-17T20:54:09.620Z,2019-12-17T20:54:09.620Z
freq,1,3,57081,185528,31337,20,20


In [4]:
# Peek at data
all_data.info()
all_data.tail(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730321 entries, 0 to 730320
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   objectId   730321 non-null  object
 1   CEP        730321 non-null  object
 2   cidade     730321 non-null  object
 3   estado     730321 non-null  object
 4   bairro     730321 non-null  object
 5   createdAt  730321 non-null  object
 6   updatedAt  730321 non-null  object
dtypes: object(7)
memory usage: 39.0+ MB


,objectId,CEP,cidade,estado,bairro,createdAt,updatedAt
730301,y9Usp35QoD,99951000,Vila Campos (Tapejara),RS - Distrito,,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z
730302,IyW5YCwCVg,99952000,Santa Cecília do Sul,RS,,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z
730303,8Z54gfYvGm,99952974,Santa Cecília do Sul,RS,Centro,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z
730304,pcWdjzOgPB,99955000,Vila Lângaro,RS,,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z
730305,cl8WHtojoM,99955971,Vila Lângaro,RS,Centro,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z
730306,xnwDF8aX46,99960000,Charrua,RS,,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z
730307,I5xskE7j6F,99960975,Charrua,RS,Centro,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z
730308,TTIR8U7H4F,99965000,Água Santa,RS,,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z
730309,VYBgItkXjj,99965971,Água Santa,RS,Centro,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z
730310,js3khuwG4r,99967000,Engenho Grande (Água Santa),RS - Distrito,,2019-12-17T21:24:00.262Z,2019-12-17T21:24:00.262Z


## Process CEP data

In [5]:
# Remove created and updated columns
all_data.drop(['createdAt', 'updatedAt'], axis=1, inplace=True)

In [6]:
# Rename columns
column_mapping = {
    'objectId': 'id',
    'CEP': 'cep',
    'cidade': 'city',
    'estado': 'state',
    'bairro': 'district'
}

all_data.rename(columns=column_mapping, inplace=True)
all_data.head()

,id,cep,city,state,district
0,b74XHFEeVf,01001000,São Paulo,SP,Sé
1,xCkm4Fn1PB,01001001,São Paulo,SP,Sé
2,ZhcbIFOMrv,01001010,São Paulo,SP,Sé
3,2fEuvsYM9J,01001900,São Paulo,SP,Sé
4,flKyzUR2tU,01001901,São Paulo,SP,Sé


In [7]:
# Create a new col containing only the first 5 digits of the zip code
all_data['zip_code_prefix'] = all_data['cep'].str[:5]

In [8]:
print(all_data['state'].unique())

['SP' 'SP  - Distrito' 'SP  - Povoado' 'RJ' 'RJ  - Distrito'
 'RJ  - Povoado' 'ES' 'ES  - Distrito' 'ES  - Povoado' 'MG'
 'MG  - Distrito' 'MG  - Povoado' 'BA' 'BA  - Distrito' 'BA  - Povoado'
 'SE' 'SE  - Distrito' 'PE' 'PE  - Povoado' 'PE  - Distrito' 'AL'
 'AL  - Povoado' 'AL  - Distrito' 'PB' 'PB  - Distrito' 'PB  - Povoado'
 'RN' 'RN  - Distrito' 'RN  - Povoado' 'CE' 'CE  - Distrito'
 'CE  - Povoado' 'PI' 'PI  - Povoado' 'MA' 'MA  - Distrito'
 'MA  - Povoado' 'PA' 'PA  - Distrito' 'PA  - Povoado' 'AP'
 'AP  - Povoado' 'AP  - Distrito' 'AM' 'AM  - Distrito' 'AM  - Povoado'
 'RR' 'RR  - Distrito' 'AC' 'AC  - Distrito' 'DF' 'GO' 'GO  - Distrito'
 'GO  - Povoado' 'RO' 'RO  - Distrito' 'RO  - Povoado' 'TO'
 'TO  - Distrito' 'TO  - Povoado' 'MT' 'MT  - Povoado' 'MT  - Distrito'
 'MS' 'MS  - Distrito' 'PR' 'PR  - Povoado' 'PR  - Distrito' 'SC'
 'SC  - Distrito' 'RS' 'RS  - Distrito' 'RS  - Povoado']


In [9]:
# Split the 'state' column by the ' - ' delimiter
all_data[['state', 'state_1']] = all_data['state'].str.split(' - ', expand=True)

In [10]:
all_data.head()

,id,cep,city,state,district,zip_code_prefix,state_1
0,b74XHFEeVf,01001000,São Paulo,SP,Sé,01001,None
1,xCkm4Fn1PB,01001001,São Paulo,SP,Sé,01001,None
2,ZhcbIFOMrv,01001010,São Paulo,SP,Sé,01001,None
3,2fEuvsYM9J,01001900,São Paulo,SP,Sé,01001,None
4,flKyzUR2tU,01001901,São Paulo,SP,Sé,01001,None


In [11]:
print(all_data['state_1'].unique())

[None 'Distrito' 'Povoado']


In [12]:
subset = ['cep', 'city', 'state']
loc_dupe = all_data.duplicated(subset = subset, keep = False)
all_data[loc_dupe].sort_values(by = 'cep')

,id,cep,city,state,district,zip_code_prefix,state_1
45510,3BTjxysVUQ,05675050,São Paulo,SP,Cidade Jardim,05675,None
45511,C40XUpM3qB,05675050,São Paulo,SP,Cidade Jardim,05675,None
45512,VhP73TXQx1,05675060,São Paulo,SP,Cidade Jardim,05675,None
45513,jQySQmJrrq,05675060,São Paulo,SP,Cidade Jardim,05675,None
45514,xKBw8qSEL0,05675070,São Paulo,SP,Cidade Jardim,05675,None
...,...,...,...,...,...,...,...
520371,tF9F1Fd6Iw,70830104,Brasília,DF,Asa Norte,70830,None
520373,PFjr1KBgFy,70830104,Brasília,DF,Asa Norte,70830,None
520375,WCsOX0ry9m,70830105,Brasília,DF,Asa Norte,70830,None
520374,WBj0JqeBmm,70830105,Brasília,DF,Asa Norte,70830,None


In [13]:
# Remove dupes
all_data = all_data.drop_duplicates(subset = subset, keep = 'first').reset_index(drop = True)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730261 entries, 0 to 730260
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               730261 non-null  object
 1   cep              730261 non-null  object
 2   city             730261 non-null  object
 3   state            730261 non-null  object
 4   district         730261 non-null  object
 5   zip_code_prefix  730261 non-null  object
 6   state_1          8052 non-null    object
dtypes: object(7)
memory usage: 39.0+ MB


In [14]:
# Inspect unique values in city (1)
city = list(all_data['city'].unique())
print('Unique city values: ', all_data['city'].nunique(), '\n', city)

Unique city values:  10162 
 ['São Paulo', 'Osasco', 'Carapicuíba', 'Barueri', 'Santana de Parnaíba', 'Pirapora do Bom Jesus', 'Jandira', 'Itapevi', 'Cotia', 'Vargem Grande Paulista', 'Taboão da Serra', 'Embu das Artes', 'Itapecerica da Serra', 'São Lourenço da Serra', 'Embu-Guaçu', 'Cipó-Guaçu (Embu-Guaçu)', 'Juquitiba', 'Guarulhos', 'Arujá', 'Santa Isabel', 'Mairiporã', 'Caieiras', 'Cajamar', 'Franco da Rocha', 'Francisco Morato', 'Ferraz de Vasconcelos', 'Poá', 'Itaquaquecetuba', 'Suzano', 'Mogi das Cruzes', 'Guararema', 'Biritiba-Mirim', 'Salesópolis', 'Nossa Senhora do Remédio (Salesópolis)', 'Santo André', 'Mauá', 'Ribeirão Pires', 'Rio Grande da Serra', 'São Caetano do Sul', 'São Bernardo do Campo', 'Diadema', 'Santos', 'Caruara (Santos)', 'Caibura (Santos)', 'Monte Cabrão (Santos)', 'Ilha Diana (Santos)', 'Bertioga', 'São Vicente', 'Guarujá', 'Cubatão', 'São Sebastião', 'Ilhabela', 'Cambaquara (Ilhabela)', 'Paranabi (Ilhabela)', 'Caraguatatuba', 'Ubatuba', 'Picinguaba (Ubatuba)

### We are only interested in the `zip_code_prefix`, so remove duplicates of these 

In [15]:
subset = ['zip_code_prefix', 'city', 'state']
loc_dupe = all_data.duplicated(subset = subset, keep = False)
all_data[loc_dupe].sort_values(by = 'cep').head(50)

,id,cep,city,state,district,zip_code_prefix,state_1
0,b74XHFEeVf,01001000,São Paulo,SP,Sé,01001,None
1,xCkm4Fn1PB,01001001,São Paulo,SP,Sé,01001,None
2,ZhcbIFOMrv,01001010,São Paulo,SP,Sé,01001,None
3,2fEuvsYM9J,01001900,São Paulo,SP,Sé,01001,None
4,flKyzUR2tU,01001901,São Paulo,SP,Sé,01001,None
5,mgbC6sblkf,01001902,São Paulo,SP,Sé,01001,None
6,uJ2DsjLMHb,01002000,São Paulo,SP,Sé,01002,None
7,71UFeITD19,01002001,São Paulo,SP,Sé,01002,None
8,9glIZw1AP4,01002010,São Paulo,SP,Sé,01002,None
9,m0FQQQjwqu,01002020,São Paulo,SP,Centro,01002,None


In [16]:
# Remove dupes
all_data = all_data.drop_duplicates(subset = subset, keep = 'last').reset_index(drop = True)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24347 entries, 0 to 24346
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               24347 non-null  object
 1   cep              24347 non-null  object
 2   city             24347 non-null  object
 3   state            24347 non-null  object
 4   district         24347 non-null  object
 5   zip_code_prefix  24347 non-null  object
 6   state_1          4910 non-null   object
dtypes: object(7)
memory usage: 1.3+ MB


In [24]:
all_data.describe()

,id,cep,city,state,district,zip_code_prefix,state_1
count,24347,24347,24347,24347,24347,24347,4910
unique,24347,24347,10162,53,7312,23369,2
top,mgbC6sblkf,01001902,São Paulo,SP,Centro,83251,Distrito
freq,1,1,3212,6302,6856,19,4186


In [25]:
subset = ['zip_code_prefix']
loc_dupe = all_data.duplicated(subset = subset, keep = False)
all_data[loc_dupe].sort_values(by = 'zip_code_prefix')

,id,cep,city,state,district,zip_code_prefix,state_1
4512,T0awRlOmAY,11200970,Caruara (Santos),SP,Caruara,11200,Povoado
4513,o3jvllfUVy,11200990,Santos,SP,,11200,None
4515,rJNZjF4j7b,11220000,Monte Cabrão (Santos),SP,,11220,Povoado
4516,JAUBaCX2lm,11220991,Santos,SP,,11220,None
5438,4AeOHtxovl,14505000,São Benedito da Cachoeirinha (Ituverava),SP,,14505,Distrito
...,...,...,...,...,...,...,...
24217,m4FxxKNFyR,99529700,Xadrez (Coqueiros do Sul),RS,,99529,Distrito
24311,8alwsXwzTY,99872000,Farrapos (São José do Ouro),RS,,99872,Distrito
24312,mUYxOOB73D,99872500,Cerro Azul (São José do Ouro),RS,,99872,Distrito
24316,vilKgTXX7t,99876500,São Miguel (São José do Ouro),RS,,99876,Distrito


#### Investigating `zip_code_prefix` duplicates, it is due to the different naming of the city. Clean these up

In [26]:
# Extract the value within parentheses (if present) or keep the original value
all_data['city1'] = all_data['city'].str.extract(r'\((.*?)\)', expand=False).fillna(all_data['city'])

In [52]:
subset = ['zip_code_prefix']
loc_dupe = all_data.duplicated(subset = subset, keep = False)
all_data[loc_dupe].sort_values(by = 'zip_code_prefix').head(20)

,id,cep,city,state,district,zip_code_prefix,state_1,city1
4512,v8SBCQm4lN,11200000,Caruara (Santos),SP,,11200,Povoado,Santos
4513,o3jvllfUVy,11200990,Santos,SP,,11200,None,Santos
4515,rJNZjF4j7b,11220000,Monte Cabrão (Santos),SP,,11220,Povoado,Santos
4516,JAUBaCX2lm,11220991,Santos,SP,,11220,None,Santos
5438,4AeOHtxovl,14505000,São Benedito da Cachoeirinha (Ituverava),SP,,14505,Distrito,Ituverava
5439,ChCB2oCFBF,14505970,Ituverava,SP,Centro,14505,None,Ituverava
5440,FuZKkGyv6E,14508000,Capivari da Mata (Ituverava),SP,,14508,Distrito,Ituverava
5441,RG06ubGFjm,14508972,Ituverava,SP,Centro,14508,None,Ituverava
5455,vHRY1rDH4F,14690000,Jurucê (Jardinópolis),SP,,14690,Distrito,Jardinópolis
5456,0EXdZNCdrW,14690970,Jardinópolis,SP,Centro,14690,None,Jardinópolis


In [27]:
all_data.drop('city', axis=1, inplace=True)

In [28]:
# Rename columns
column_mapping = {
    'city1': 'city'
}

all_data.rename(columns=column_mapping, inplace=True)
all_data.head()

,id,cep,state,district,zip_code_prefix,state_1,city
0,mgbC6sblkf,01001902,SP,Sé,01001,None,São Paulo
1,6slRyhnKbB,01002903,SP,Sé,01002,None,São Paulo
2,m6VFkRepYy,01003905,SP,Sé,01003,None,São Paulo
3,Pb3iRaHvLJ,01004904,SP,Sé,01004,None,São Paulo
4,EEBlPPk6tg,01005900,SP,Sé,01005,None,São Paulo


In [29]:
subset = ['zip_code_prefix']
loc_dupe = all_data.duplicated(subset = subset, keep = False)
all_data[loc_dupe].sort_values(by = 'zip_code_prefix').head(20)

,id,cep,state,district,zip_code_prefix,state_1,city
4512,T0awRlOmAY,11200970,SP,Caruara,11200,Povoado,Santos
4513,o3jvllfUVy,11200990,SP,,11200,None,Santos
4515,rJNZjF4j7b,11220000,SP,,11220,Povoado,Santos
4516,JAUBaCX2lm,11220991,SP,,11220,None,Santos
5438,4AeOHtxovl,14505000,SP,,14505,Distrito,Ituverava
5439,ChCB2oCFBF,14505970,SP,Centro,14505,None,Ituverava
5440,FuZKkGyv6E,14508000,SP,,14508,Distrito,Ituverava
5441,RG06ubGFjm,14508972,SP,Centro,14508,None,Ituverava
5455,vHRY1rDH4F,14690000,SP,,14690,Distrito,Jardinópolis
5456,0EXdZNCdrW,14690970,SP,Centro,14690,None,Jardinópolis


In [30]:
# Remove dupes
all_data = all_data.drop_duplicates(subset = subset , keep = 'last').reset_index(drop = True)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23369 entries, 0 to 23368
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               23369 non-null  object
 1   cep              23369 non-null  object
 2   state            23369 non-null  object
 3   district         23369 non-null  object
 4   zip_code_prefix  23369 non-null  object
 5   state_1          3981 non-null   object
 6   city             23369 non-null  object
dtypes: object(7)
memory usage: 1.2+ MB


In [31]:
all_data.describe()

,id,cep,state,district,zip_code_prefix,state_1,city
count,23369,23369,23369,23369,23369,3981,23369
unique,23369,23369,53,7270,23369,2,5269
top,mgbC6sblkf,01001902,SP,Centro,01001,Distrito,São Paulo
freq,1,1,6302,6530,1,3641,3212


In [32]:
# Remove created and updated columns
all_data.drop('state_1', axis=1, inplace=True)

In [23]:
# Random check on city names
result = all_data[all_data['city'].str.contains('Barbacena')]

print(result)

Empty DataFrame
Columns: [id, cep, city, state, district, zip_code_prefix, state_1]
Index: []


In [33]:
all_data.head(50)

,id,cep,state,district,zip_code_prefix,city
0,mgbC6sblkf,01001902,SP,Sé,01001,São Paulo
1,6slRyhnKbB,01002903,SP,Sé,01002,São Paulo
2,m6VFkRepYy,01003905,SP,Sé,01003,São Paulo
3,Pb3iRaHvLJ,01004904,SP,Sé,01004,São Paulo
4,EEBlPPk6tg,01005900,SP,Sé,01005,São Paulo
5,vCS4KLUvSp,01006904,SP,Sé,01006,São Paulo
6,VBEs9t9Y5A,01007906,SP,Sé,01007,São Paulo
7,hmNd6eeudm,01008908,SP,Centro,01008,São Paulo
8,WHiXJS3zr9,01009999,SP,Centro,01009,São Paulo
9,dw48UYMaq3,01010905,SP,Centro,01010,São Paulo


## Export the cleaned CEP data to csv

In [92]:
# Export to .csv file
from datetime import date
today = date.today()
all_data.to_csv(f'{export_path_local}cep_{today}.csv')

## Merge with processed geolocation dataset to push in proper names
Import processed geolocation dataset, then merge

In [34]:
# Import geolocation data to merge
geo_df = pd.read_csv(f'{export_path_local}geolocation_2024-04-27.csv')

In [35]:
geo_df.describe()

,Unnamed: 0,zip_code_prefix,lat,lng
count,720154.000000,720154.000000,720154.000000,720154.000000
mean,360076.500000,38611.140120,-20.974809,-46.452045
std,207890.697216,30576.575761,5.918866,4.419726
min,0.000000,1001.000000,-36.605374,-101.466766
25%,180038.250000,13043.000000,-23.601862,-48.912225
50%,360076.500000,29345.000000,-22.861875,-46.645727
75%,540114.750000,66053.000000,-19.916325,-43.786507
max,720153.000000,99990.000000,45.065933,121.105394


In [36]:
geo_df.head()

,Unnamed: 0,zip_code_prefix,lat,lng,city,state
0,0,1001,-23.549292,-46.633559,sao paulo,SP
1,1,1001,-23.550498,-46.634338,sao paulo,SP
2,2,1001,-23.550642,-46.634410,sao paulo,SP
3,3,1001,-23.549698,-46.633909,sao paulo,SP
4,4,1001,-23.550263,-46.634196,sao paulo,SP


In [37]:
# Zip code prefix should be 5 digits long with leading zeroes
geo_df['zip_code_prefix'] = geo_df['zip_code_prefix'].apply(lambda x: '{0:0>5}'.format(x))

In [38]:
# Remove old index col
geo_df.drop('Unnamed: 0', axis=1, inplace=True)

In [39]:
# Merge geo_df with zip with a 'left join'
merged_df = geo_df.merge(all_data, how='left', left_on='zip_code_prefix', right_on='zip_code_prefix')

In [40]:
display(merged_df[merged_df['zip_code_prefix']=='73010'])

,zip_code_prefix,lat,lng,city_x,state_x,id,cep,state_y,district,city_y
560707,73010,-15.647606,-47.787974,brasilia,DF,p6hUVUlzwZ,73010901,DF,Sobradinho,Brasília
560708,73010,-15.646190,-47.789848,brasilia,DF,p6hUVUlzwZ,73010901,DF,Sobradinho,Brasília
560709,73010,-15.647701,-47.790285,brasilia,DF,p6hUVUlzwZ,73010901,DF,Sobradinho,Brasília
560710,73010,-15.647891,-47.788036,brasilia,DF,p6hUVUlzwZ,73010901,DF,Sobradinho,Brasília
560711,73010,-15.651576,-47.793778,brasilia,DF,p6hUVUlzwZ,73010901,DF,Sobradinho,Brasília
560712,73010,-15.646750,-47.787791,brasilia,DF,p6hUVUlzwZ,73010901,DF,Sobradinho,Brasília
560713,73010,-15.648747,-47.788220,brasilia,DF,p6hUVUlzwZ,73010901,DF,Sobradinho,Brasília
560714,73010,-15.652373,-47.790856,brasilia,DF,p6hUVUlzwZ,73010901,DF,Sobradinho,Brasília


In [41]:
# Replace city_x with city_y only if city_y exists
merged_df['city_x'] = merged_df['city_y'].combine_first(merged_df['city_x'])

In [42]:
# Replace city_x with city_y only if city_y exists
merged_df['state_x'] = merged_df['state_y'].combine_first(merged_df['state_x'])

In [43]:
# Remove created and updated columns
merged_df.drop(['state_y', 'city_y', 'id'], axis=1, inplace=True)

In [44]:
# Rename columns
column_mapping = {
    'city_x': 'city',
    'state_x': 'state'
}

merged_df.rename(columns=column_mapping, inplace=True)
merged_df.head()

,zip_code_prefix,lat,lng,city,state,cep,district
0,01001,-23.549292,-46.633559,São Paulo,SP,01001902,Sé
1,01001,-23.550498,-46.634338,São Paulo,SP,01001902,Sé
2,01001,-23.550642,-46.634410,São Paulo,SP,01001902,Sé
3,01001,-23.549698,-46.633909,São Paulo,SP,01001902,Sé
4,01001,-23.550263,-46.634196,São Paulo,SP,01001902,Sé


In [45]:
# Check for duplicates based on location info only
subset = ['zip_code_prefix', 'lat', 'lng']
loc_dupe = merged_df.duplicated(subset = subset, keep = False)
merged_df[loc_dupe].sort_values(by = 'lat')

,zip_code_prefix,lat,lng,city,state,cep,district


In [46]:
# Inspect unique values in city (1)
city = list(merged_df['city'].unique())
print('Unique city values: ', merged_df['city'].nunique(), '\n', city)

Unique city values:  5497 
 ['São Paulo', 'sao paulo', 'Osasco', 'osasco', 'Carapicuíba', 'carapicuiba', 'Barueri', 'barueri', 'Santana de Parnaíba', 'Pirapora do Bom Jesus', 'Jandira', 'jandira', 'itapevi', 'Itapevi', 'Cotia', 'Vargem Grande Paulista', 'vargem grande paulista', 'Taboão da Serra', 'Embu das Artes', 'Itapecerica da Serra', 'São Lourenço da Serra', 'Embu-Guaçu', 'Juquitiba', 'Guarulhos', 'guarulhos', 'Arujá', 'Santa Isabel', 'Mairiporã', 'Caieiras', 'caieiras', 'Cajamar', 'jordanesia', 'polvilho', 'cajamar', 'Franco da Rocha', 'Francisco Morato', 'Ferraz de Vasconcelos', 'Poá', 'Itaquaquecetuba', 'Suzano', 'suzano', 'Mogi das Cruzes', 'Guararema', 'Biritiba-Mirim', 'Salesópolis', 'Santo André', 'santo andre', 'Mauá', 'maua', 'Ribeirão Pires', 'Rio Grande da Serra', 'São Caetano do Sul', 'São Bernardo do Campo', 'sao bernardo do campo', 'Diadema', 'Santos', 'santos', 'Bertioga', 'São Vicente', 'sao vicente', 'Guarujá', 'Cubatão', 'São Sebastião', 'maresias', 'Ilhabela', '

In [129]:
# Convert all city names to tltle case (init caps)
merged_df['city'] = merged_df['city'].str.title()

In [19]:
# Randome check presence of city name
result = merged_df[merged_df['city'].str.contains('Uba')]

print(result)

NameError: name 'merged_df' is not defined

## Export the merged file to csv

In [121]:
# Export to .csv file
from datetime import date
today = date.today()
merged_df.to_csv(f'{export_path_local}geolocation_{today}.csv')

# END